<a href="https://colab.research.google.com/github/elizathornton/Provider-Metrics-Dashboard/blob/main/dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

!pip install jupyter-dash
!pip install pyngrok

from jupyter_dash import JupyterDash
import dash
from dash import Dash, dcc, html, Input, Output
from pyngrok import ngrok

import string

# from dateutil import parser
# !pip install -U arrow
# import arrow
# from datetime import datetime
# !pip install streamlit

# !pip install anvil-uplink
# import anvil.server
# anvil.server.connect("")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

## scheme - we import data frames for all 5 values, where the index is providers
# we  delete any office data, and combine duplicate providers within each dataframe
# we combine all the data frames by providers  
# anywhere there is a null or NaN we substitute a zero

# ---- RVU Dataframe ------  BASICALLY DONE
# This block will read all the CSV files and create the data frames for the ap to work on
# rvudata will be a data frame with providers vs RVU/Day for each week. 

# I am combining all the excel spreadsheets manually right now. 

#  RVU DATA - all numeric - no columns with strings

rvudata = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/dash_rvu.csv')
rvudata = rvudata.dropna(axis=1, how='all')



# Look for all the instances of "RVU/Day" and  look 2 to the left and 2 up  to find "Wk End _____"
# Change the name of RVU/Day column to the week end date, and make it a date data type
# Then we will delete all the rows above the header row and make that row our header
# Now create a new column for SUM of FTE counts, SUM of direct rvu, and 
# then AVERAGE RVU / Day across the entire timeframe, call it something indicating the time frame you are averaging
headrow = rvudata.iloc[4]

for i in range(rvudata.shape[1]):
  if headrow.iloc[i] == 'RVU/Day':
    x = rvudata.iloc[2,i-2]      #  x is now for example "Wk End 6/19/2022"
    start = x.find('Wk End')
    wkend = x[start+6:].strip().split(' ')[0] #extract just 6/19/2022 from x
 #   print(wkend)
    rvudata.iloc[4,i]=wkend
    rvudata.iloc[4,i-1]='RVU/Week '+str(wkend)
    rvudata.iloc[4,i-2]='FTE '+str(wkend)
    


headrow = rvudata.iloc[4]
rvudata.drop(rvudata.index[0:5], inplace=True) # cut out the top rows
rvudata.columns = headrow      # make the column headers the labels saved

dlist = ['Office','Prov Code']
rvudata.drop(dlist, axis=1, inplace=True)   # delete the office, prov code

rvudata['Provider'] = rvudata['Provider'].str.upper()  # make sure providers are all capitalized
rvudata.set_index('Provider', inplace = True) #povider column is now index

rvudata = rvudata.replace([' '],'0') #replace blank spaces with 0
rvudata = rvudata.apply(pd.to_numeric, errors = "coerce")  #change all the values from string to numbers
rvudata = rvudata.groupby(rvudata.index, axis=0).sum() # when a provider has some rvus in multiple places this sums them into a single column

rvudata['FTE Sum'] = 0
rvudata['RVU Sum'] = 0
rvudata['Average RVU/Day'] = 0


for i in range(rvudata.shape[1]):
  if 'RVU/Week' in rvudata.columns[i]:

    fte_label = rvudata.columns[i-1]
    rvu_label = rvudata.columns[i]
    rvudata['FTE Sum'] = rvudata['FTE Sum'] + rvudata[fte_label]
    rvudata['RVU Sum'] = rvudata['RVU Sum'] + rvudata[rvu_label]


rvudata['Average RVU/Day'] = rvudata['RVU Sum']/(5*rvudata['FTE Sum'])

dlist = [col for col in rvudata.columns if 'FTE' in col or 'RVU/Week' in col or 'Sum'in col]
rvudata.drop(dlist, axis=1, inplace=True)   # delete the  FTE, RVU columns

rvudata = rvudata.fillna(0)
rvudata = rvudata[rvudata.index.str.contains("Office|OFFICE|Location|LOCATION") == False]

quartlist = ['Quarter 2']
rvudata.head()


4,6/19/2022,6/12/2022,6/5/2022,5/29/2022,5/22/2022,5/15/2022,5/8/2022,5/1/2022,4/24/2022,4/17/2022,4/10/2022,4/3/2022,3/27/2022,3/20/2022,3/13/2022,3/6/2022,Average RVU/Day
Provider,,,,,,,,,,,,,,,,,
"ALASTOR MOODY, MD",42.58,44.40,40.20,48.40,40.6,38.2,37.40,32.6,22.8,57.6,52.4,41.6,44.6,40.2,33.4,44.4,41.320380
"ALBUS DUMBLEDORE, NP",26.91,26.04,38.07,36.68,53.4,46.4,44.20,51.0,34.2,46.6,59.8,22.8,26.4,46.4,23.8,39.8,38.430375
"ARABELLA FIGG, NP",31.28,22.82,23.72,27.91,22.2,31.0,23.60,23.8,24.2,26.2,32.6,35.4,35.6,37.8,26.4,24.6,27.837089
"BELLATRIX LESTRANGE, NP",0.00,0.00,0.00,0.00,0.0,0.0,26.45,22.2,31.0,23.6,23.8,24.2,26.2,32.6,22.8,26.4,25.902917
"GILDEROY LOCKHART, PA",26.60,30.20,23.60,23.80,24.2,26.2,32.60,22.8,26.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.266667


In [ ]:
#  --------- HCC dataframe ------  BASICALLY DONE
# Read in the HCC spreadsheet
# Get to a spreadsheet that has providers as index and HCC as single column
# Combine (average?) for any duplications of provider names
# Sort alphabetically by last name
# compare the provider list to the previous
# create a provider called "Mean for all providers" and make that the first in the provider list. 


hccdata = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/dash_hcc.csv')
hccdata = hccdata.dropna(axis=1, how='all')    # get rid of columns with all null
hccdata.rename(columns = {'AssignedVPAProvider':'Provider', 'Recapture_Rate':'Recapture Rate'}, inplace = True)
hccdata['Provider'] = hccdata['Provider'].str.upper()  # make sure providers are all capitalized
hccdata.set_index('Provider', inplace = True) #povider column is now index
hccdata = hccdata[['Recapture Rate']]  #drop all the other columns - just keep recapture rate


hccdata = hccdata[hccdata.index.str.contains("Office|OFFICE|Location|LOCATION") == False]


hccdata.replace({'%': ''}, regex=True, inplace=True)   # Get rid of the % sign
hccdata = hccdata.apply(pd.to_numeric, errors = "coerce")  #change all the values from string to numbers
hccdata = hccdata.groupby(hccdata.index, axis=0).mean() # when a provider has some metrics in multiple places this averages them into a single row

hccdata = hccdata.fillna(0)


hccdata.head()


,Recapture Rate
Provider,
"BLACK, SIRIUS",53.61
"DUMBLEDORE, ALBUS",55.62
"EVANS POTTER, LILY",59.94
"FIGG, ARABELLA",62.81
"FINCH-FLETCHLEY, JUSTIN",59.44


In [ ]:

# percent reduction in hospital admission -  don't have data, put a dummy chart there
# BASICALLY DONE


eddata = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/dash_ed.csv')
eddata = eddata.dropna(axis=1, how='all')    # get rid of columns with all null
#eddata.columns  = eddata.iloc[0]  # make the head row the first row of the spreadsheet

eddata['Provider'] = eddata['Provider'].str.upper()  # make sure providers are all capitalized
eddata.set_index('Provider', inplace = True)

eddata = eddata[eddata.index.str.contains("Office|OFFICE|Location|LOCATION") == False]


eddata = eddata.apply(pd.to_numeric, errors = "coerce")  #change all the values from string to numbers
eddata = eddata.groupby(eddata.index, axis=0).mean() # when a provider has some metrics in multiple places this averages them into a single row

eddata = eddata.fillna(0)

eddata.head()

,Percent Reduction in ED use
Provider,
"RUBEUS HAGRID, NP",4.0
"ALBUS DUMBLEDORE, PA",4.0
"ARABELLA FIGG, NP",4.0
"BELLATRIX LESTRANGE, NP",4.0
"GILDEROY LOCKHART, MD",4.0


In [ ]:


#quality measures - will have gauge graph  and will also have a bar graph
# bar graph percent on y axis, measure (eg. colonosocopies, bp control) on x axis
# BASICALLY DONE

qualdata = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/dash_qual.csv')
qualdata = qualdata.dropna(axis=1, how='all')    # get rid of columns with all null
qualdata = qualdata.T   # transpose it so the measures are columns and providers are rows

qualdata.iloc[0,1] = 'Overall'
qualdata.iloc[0,2] = 'Preventative'
qualdata.iloc[0,10] = 'Diabetes'
qualdata.iloc[0,12] = 'Cardio'
qualdata.iloc[0,14] = 'Other'

qualdata.columns  = qualdata.iloc[0]
qualdata = qualdata.iloc[1:]
qualdata = qualdata[qualdata.index.str.contains("Office|OFFICE|Location|LOCATION") == False]


qualdata.replace({'%': ''}, regex=True, inplace=True)   # Get rid of the % sign
qualdata = qualdata.apply(pd.to_numeric, errors = "coerce")  #change all the values from string to numbers
qualdata = qualdata.groupby(qualdata.index, axis=0).mean() # when a provider has some metrics in multiple places this averages them into a single row

qualdata = qualdata.fillna(0)

qualdata.head()



Provider,Ranking,Overall,Preventative,Colorectal CA Screening,Tobacco Use NP Screening,Tobacco Use NP Intervention,Tobacco Use NP Screening & Intervention,Statin Therapy for Prev & Tx of CVD,Clinical Depression Screen and F/U Plan,Mammography Screening in Women,Diabetes,DM: HbA1c Control (<9%) NP GPRO,Cardio,Controlling High BP,Other,Falls: Screening for Future Fall Risk,Depression Remission at Twelve Months NP GPRO
"Black, Sirius MD",1.0,94.0,94.3,11.8,100.0,100.0,100.0,93.0,92.6,0.0,20.0,20.0,91.7,91.7,95.7,95.7,0.0
"Evans Potter, Lily MD",4.0,91.7,91.4,62.5,100.0,100.0,100.0,94.5,97.8,29.4,89.5,89.5,85.0,85.0,97.7,97.7,0.0
"Figg, Arabella",5.0,91.4,92.2,68.4,98.1,100.0,98.1,97.9,82.6,46.7,66.7,66.7,92.9,92.9,91.3,91.3,0.0
"Finch-Fletchley, Justin MD",6.0,90.9,92.6,58.8,99.2,100.0,99.2,86.7,85.0,50.0,80.0,80.0,83.3,83.3,86.4,86.4,0.0
"Flamel, Nicolas NP",8.0,89.3,90.0,30.0,100.0,100.0,100.0,60.0,84.7,9.1,81.8,81.8,91.3,91.3,86.3,86.3,0.0


In [ ]:

# PRess ganey -  BASICALLY DONE
# gauge graph and then 2 bar graphs - 1 for individual, 1 for office
# Question        1 - Overall Access  
#            2 - Ease of Scheduling Appointments   
#           3 - Likelihood of Recommending a Care Provider   
#           4 - Staff Worked Together to Care for You
survdata = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/dash_surv.csv')
survdata = survdata.dropna(axis=1, how='all')    # get rid of columns with all null


for i in range(survdata.shape[1]):
  if 'Access Overall' in str(survdata.iloc[2,i]):
    survdata.iloc[5,i] = 'Access Overall'
  elif 'Ease of Scheduling Appointments' in str(survdata.iloc[2,i]):
    survdata.iloc[5,i] = 'Ease of Scheduling Appointments'
  elif 'Likelihood of Recommending a Care Provider' in str(survdata.iloc[2,i]):
    survdata.iloc[5,i] = 'Likelihood of Recommending a Care Provider'
  elif 'Staff Worked Together to Care for You' in str(survdata.iloc[2,i]):
    survdata.iloc[5,i] = 'Staff Worked Together to Care for You'

headrow = survdata.iloc[5]
#print (headrow)
survdata.drop(survdata.index[0:6], inplace=True) # cut out the top rows
survdata.columns = headrow 

# Not going to keep the office column right now b/c I need all my values to be numeric, so  I can graph them
clist = ['Provider','Access Overall','Ease of Scheduling Appointments','Likelihood of Recommending a Care Provider','Staff Worked Together to Care for You','Experience of Care (Weighted Average)']
survdata = survdata[clist]
survdata['Provider'] = survdata['Provider'].str.upper()
survdata.rename(columns = {'Experience of Care (Weighted Average)':'Experience of Care'})
survdata.set_index('Provider', inplace = True)


survdata.replace({'%': ''}, regex=True, inplace=True)   # Get rid of the % sign
survdata = survdata.apply(pd.to_numeric, errors = "coerce")  #change all the values from string to numbers
survdata = survdata.groupby(survdata.index, axis=0).mean() # when a provider has some metrics in multiple places this averages them into a single row

survdata = survdata.fillna(0)

survdata = survdata[survdata.index.str.contains("Office|OFFICE|Location|LOCATION") == False]


survdata.head()

5,Access Overall,Ease of Scheduling Appointments,Likelihood of Recommending a Care Provider,Staff Worked Together to Care for You,Experience of Care (Weighted Average)
Provider,,,,,
ALASTOR MOODY MD,85.00,80.00,85.00,85.00,3.5
ALBUS DUMBLEDORE NP,91.67,91.67,100.00,100.00,5.0
ARABELLA FIGG NP,94.64,96.43,89.29,92.86,5.0
BELLATRIX LESTRANGE NP,92.19,87.50,96.88,96.88,5.0
GILDEROY LOCKHART MD,90.63,89.58,90.38,91.67,5.0


In [ ]:
# Function to CLEAN UP the provider names - at this point should get passed 1 values
# Either Doe or Doe-ray or Jane or JANE, or JANE-MARY
# caps and commas could be any:  DOE-RAY,  or doe-ray  
def clean_name(name):
  name = name.replace(",","")
  name = name.strip()
  if '-' in name:
    j = name.split('-')
    name = string.capwords(j[0])+'-'+string.capwords(j[1])
  else:
    name = string.capwords(name)
  
  return name
  


In [ ]:
# FUNCTION to standardize the provider names
# Can be in format:  Jane Doe, MD      Jane Doe MD    Doe, Jane      Doe, Jane, MD       Doe, Jane MD     Jane Doe  
# Might have 3 names
# Will standardize it to this form:  Doe, Jane
# Remove the tag (MD DO NP CRNP PA) - tag should be at the end.
# Remove trailing comma or white space
# Split it with a space
# Make sure there are only 2 names 
# If there is no comma label the first one the first name and the last one the last name
# If there is a comma  -> other way around and remove the comma
# Reconstruct the name in a standard format like Doe, Jane
# Alphabetize by last name

# Create a new provider named "Mean Provider Values", make it the first provider in each data frame, and make the values be the mean, excluding anyone who has all zeros

def standardize_name(df):

  degrees = ['MD','DO','NP','CRNP','CNP','PA']

  provider_list = df.index.tolist()

  for i in range(len(provider_list)):
    name = str(provider_list[i])   # This is the original contents of provider name, could be with or without degree
    deg = ''.join(set(name.split())&set(degrees))   # this is a string which returns a degree if present
    if deg: 
      name = name.replace(deg,'')  # if there is a degree in the name, get rid of it now
      name = name.strip() # pull spaces off the end of the name
      if (name[len(name)-1] ==','): name = name[:-1] #get rid of the comma if there is one. 

    name = name.split()   # split by spaces - will either be 2 or 3 names - name WAS a string, NOW  a list
    
    firstname = ''
    lastname = ''
    if len(name) == 2:  # Jane Doe or Doe, Jane 
      if ',' in name[0]:     # Doe, Jane (comma in first part) - means the last name is first
        lastname = clean_name(name[0])
        firstname = clean_name(name[1])
      else:                  # 2 words, no comma - Jane Doe - means first name is first
        firstname = clean_name(name[0])
        lastname = clean_name(name[1])

    elif len(name) == 3:  # Jane Doe Ray OR Doe Ray, Jane OR Jane Jill Doe OR Doe, Jane Jill
      # There is no way to distinguish between Jane Doe Ray and Jane Jill Doe, so if there are 3 names
      # with no commas then we will assume it is 2 last names (Jane Doe Ray)
      if ',' in name[0]:   # Doe, Jane Jill
        firstname = clean_name(name[1]+'-'+name[2])
        lastname = clean_name(name[0])
      if ',' in name[1]:   # Doe Ray, Jane
        firstname = clean_name(name[2])
        lastname = clean_name(name[0]+'-'+name[1])
      else:
        firstname = clean_name(name[0])
        lastname = clean_name(name[1]+'-'+name[2])
    else:
      pass          # if there are not 2 or 3 names, not sure what the situation is so just pass


    if (firstname != '' and lastname !=''):
      provider_list[i] = lastname+', '+firstname

  df.index = provider_list

  df = df.apply(pd.to_numeric, errors = "coerce")
  df = df.replace(0, np.NaN)
  df.loc['Mean Values'] = df.mean()  # create a row of mean values
  df = df.fillna(0).sort_index()

  return df

    


In [ ]:

# Merge all the provider lists and delete duplicates to make a master provider list
# For each data frame, check that each provider has data - if not, add that provider and just put all zeros

# First, standardize all the data frame provider names to Doe, Jane

rvudata = standardize_name(rvudata)
hccdata = standardize_name(hccdata)
eddata = standardize_name(eddata)
qualdata = standardize_name(qualdata)
survdata = standardize_name(survdata)

print(rvudata.head())
print(hccdata.head())
print(eddata.head())
print(qualdata.head())
print(survdata.head())

4                        6/19/2022  6/12/2022  6/5/2022  5/29/2022  5/22/2022  \
Black, Sirius                22.20      31.00     23.60      23.80      24.20   
Dumbledore, Albus            26.91      26.04     38.07      36.68      53.40   
Evans-Potter, Lily           33.54      32.84     34.67      34.59      34.59   
Figg, Arabella               31.28      22.82     23.72      27.91      22.20   
Finch-Fletchley, Justin      59.05      27.88     22.70      48.27      48.27   

4                        5/15/2022  5/8/2022  5/1/2022  4/24/2022  4/17/2022  \
Black, Sirius                 26.2      32.6      22.8       26.4       19.6   
Dumbledore, Albus             46.4      44.2      51.0       34.2       46.6   
Evans-Potter, Lily            33.2      32.6      36.6       35.2       37.6   
Figg, Arabella                31.0      23.6      23.8       24.2       26.2   
Finch-Fletchley, Justin       22.2      31.0      23.6       23.8       24.2   

4                        4/10/20

In [ ]:
# Now compare all the provider lists, make a master list with ALL the names 
# Then append all the data frames with any provider names that aren't in their 
# original data frame, and make the values zero


srvu = set(rvudata.index.values.tolist()) 
shcc = set(hccdata.index.values.tolist()) 
sed = set(eddata.index.values.tolist()) 
squal = set(qualdata.index.values.tolist()) 
ssurv = set(survdata.index.values.tolist()) 

masterset = srvu | shcc | sed | squal | ssurv
print(len(masterset))

try:
  alist = list(masterset - srvu)
  column_names = rvudata.columns
  df2 = pd.DataFrame(index=alist,columns = column_names)
  rvudata = rvudata.append(df2).fillna(0).sort_index()
except: pass

try:
  alist = list(masterset - shcc)
  column_names = hccdata.columns
  df2 = pd.DataFrame(index=alist,columns = column_names)
  hccdata = hccdata.append(df2).fillna(0).sort_index()
except: pass

try:
  alist = list(masterset - sed)
  column_names = eddata.columns
  df2 = pd.DataFrame(index=alist,columns = column_names)
  eddata = eddata.append(df2).fillna(0).sort_index()
except: pass

try:
  alist = list(masterset - squal)
  column_names = qualdata.columns
  df2 = pd.DataFrame(index=alist,columns = column_names)
  qualdata = qualdata.append(df2).fillna(0).sort_index()
except: pass

try:
  alist = list(masterset - ssurv)
  column_names = survdata.columns
  df2 = pd.DataFrame(index=alist,columns = column_names)
  survdata = survdata.append(df2).fillna(0).sort_index()
except: pass

print(rvudata.shape)

print(hccdata.shape)

print(eddata.shape)

print(qualdata.shape)

print(survdata.shape)




25
(25, 17)
(25, 1)
(25, 1)
(25, 17)
(25, 5)


In [ ]:
# make a master provider list
providers = rvudata.index.values.tolist()
print(providers)




['Black, Sirius', 'Dumbledore, Albus', 'Evans-Potter, Lily', 'Figg, Arabella', 'Finch-Fletchley, Justin', 'Flamel, Nicolas', 'Fletcher, Mundungus', 'Granger, Hermione', 'Hagrid, Rubeus', 'Lestrange, Bellatrix', 'Lockhart, Gilderoy', 'Longbottom, Neville', 'Lovegood, Luna', 'Lupin, Remus', 'Mcgonagall, Minerva', 'Mean Values', 'Moody, Alastor', 'Potter, Harry', 'Riddle, Tom', 'Skeeter, Rita', 'Snape, Severus', 'Tonks, Nymphadora', 'Tonks, Nyphadora', 'Trelawney, Sybill', 'Weasley, Ronald']


In [ ]:
# FUNCTION to make the gauge graphs
# each gauge graph will have 3 basic colors - red, yellow, green.  
# The cutoffs will be in an list  - for example rvu below 38.4 is red, above 43.2 is green and in between is yellow
# The list will start with the lowest number displayed on the chart, then the cutoff for yellow (min for any bonus),
# then the cuttoff for green (100% bonus), then the highest number displayed on chart

def gaugeplot(bar,cuts,title):


  
  if bar < cuts[1]:
    barcolor = "red"
  elif bar > cuts[2]:
    barcolor = "green"
  else:
    barcolor = "yellow"


  gauge = go.Figure(go.Indicator(
    mode = "gauge+number",
    value = bar,
    domain = {'x': [0, 1], 'y': [0, 1]},
    gauge = {'axis': {'range': [cuts[0], cuts[3]]},
            'bar': {'color': barcolor},
             'steps' : [
                 {'range': [cuts[0], cuts[1]], 'color': 'rgba(128,0,0,0.2)'},  # faint red, yellow, green
                 {'range': [cuts[1], cuts[2]], 'color': 'rgba(128,128,0,0.2)'},
                 {'range': [cuts[2], cuts[3]], 'color': 'rgba(0,128,0,0.2)'}]},
    title = {'text': title},
    ),
    go.Layout(
          margin = dict(l=30,r=30,b=0,t=0)
    ))

  return gauge


In [ ]:
# This block of code reads the cutoff levels for the bonuses
# values in the first column represent the measure level, 2nd column represent the percent bonus 
rvucuts = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/Measure Bonus Cutoffs - RVU.csv')
hcccuts = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/Measure Bonus Cutoffs - HCC.csv')
qualcuts = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/Measure Bonus Cutoffs - Quality.csv')
edcuts = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/Measure Bonus Cutoffs - ED.csv')
survcuts  = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/Measure Bonus Cutoffs - Survey.csv')

rvucuts = rvucuts.apply(pd.to_numeric, errors = "coerce") 
hcccuts = hcccuts.apply(pd.to_numeric, errors = "coerce")
edcuts = edcuts.apply(pd.to_numeric, errors = "coerce")
qualcuts = qualcuts.apply(pd.to_numeric, errors = "coerce")
survcuts = survcuts.apply(pd.to_numeric, errors = "coerce")
# Now lets make a dataframe of the cuts to pass to the gauge plot function
# The list will start with the lowest number displayed on the chart, then the cutoff for yellow (min for any bonus),
# then the cuttoff for green (100% bonus), then the highest number displayed on chart




In [ ]:
# This little function takes a cuts dataframe (from the spreadsheet listing bonus levels)
# and returns a list of 4 numbers:
# The lowest number displayed on the chart, = 1/2 of min bonus metric 
# then the cutoff for yellow = min for any bonus
# then the cuttoff for green = 100% bonus)
# then the highest number displayed on chart = last number from df (either 100 or 120 % bonus or whatever is highest)

def gauge_tick_measures(df):

  blist = list(df['Bonus'])
  fullbonuslevel = blist.index(100)
  tick1 = df['Metric'].iloc[0]*0.5
  tick2 = df['Metric'].iloc[0]
  tick3 = df['Metric'].iloc[fullbonuslevel]
  tick4 = df['Metric'].iloc[len(blist)-1]

  alist = [tick1,tick2,tick3,tick4]
#  print(alist)
  return alist


In [ ]:

# external CSS stylesheets
external_stylesheets = [
    'https://codepen.io/chriddyp/pen/bWLwgP.css',
    {
        'href': 'https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/css/bootstrap.min.css',
        'rel': 'stylesheet',
        'integrity': 'sha384-MCw98/SFnGE8fJT3GXwEOngsV7Zt27NXFoaoApmYm81iuXoPkFOJwJ8ERdknLPMO',
        'crossorigin': 'anonymous'
    }
]


#app = Dash(__name__, external_stylesheets=external_stylesheets)


#external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

app.layout = html.Div([

 # html.H1('Provider Metrics', 
    #        style = {'text-align': 'center','position': 'sticky','z-index':1, 'top': '0px'}
     #  ),


    # Start of the 2 drop down menus
    html.Div([
        html.H1('Provider Metrics', 
            style = {'text-align': 'center', 'font-size' : '400%'}),
        html.Div([
            dcc.Dropdown(
                id='provider_list',
                options=[{'label': i, 'value': i} for i in providers],
                value='Mean Values' 
                #CHANGE - put in here the mean for all providers later as default val
            )
        ],
        style={'width': '70%', 'height':'100%', 'display': 'inline-block','font-size' : '300%'}),

        html.Div([
            dcc.Dropdown(
                id='quarter',
                options=[{'label': i, 'value': i} for i in quartlist],
                value='Quarter 2'
            )
        ], style={'width': '25%', 'float': 'right', 'display': 'inline-block','font-size' : '300%'})

    ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'  #,'position': 'sticky','z-index':10000, 'top': '0px'
    }),
# End of the 2 drop down menus


# Top row Side by side Graphs
    html.Div([
      html.Div([
          dcc.Graph(id='hcc_gauge'),
      ], style={'display': 'inline-block', 'width': '33%'}),  

      html.Div([
          dcc.Graph(id='rvu_gauge'),
      ], style={'display': 'inline-block', 'width': '33%'}), 

      html.Div([
          dcc.Graph(id='ed_gauge'),
      ], style={'display': 'inline-block', 'width': '33%'})
    ]),

# bottom row Side by side Graphs
    html.Div([
      html.Div([
          dcc.Graph(id='qual_gauge'),
      ], style={'display': 'inline-block', 'width': '33%'}),

      html.Div([
          dcc.Graph(id='survey_gauge'),
      ], style={'display': 'inline-block', 'width': '33%'}),

      html.Div([
 #         dcc.Graph(id='survey_gauge'),
      ], style={'display': 'inline-block', 'width': '33%'})
    ]),
# End of side by side graphs
# STart of long graphs
    html.Div([
      dcc.Graph(id='rvu_bar') ],                          
      style={'display': 'inline-block'}  ),

    html.Div([
      dcc.Graph(id='qual_bar') ],                          
      style={'display': 'inline-block'}  ),

    html.Div([
      dcc.Graph(id='surv_bar') ],                         
      style={'display': 'inline-block'}  ), 
# End of long graphs
    html.Div([
     dcc.Textarea(
        id="testout",
        value=" ",
        style={"height": 100},
    )])

],  # style={'background-color': 'rgba(0,128,128,0.3)'}
)


#update the rvu gauge chart
@app.callback([
  Output('rvu_bar', 'figure'),
  Output('qual_bar', 'figure'),
  Output('surv_bar', 'figure'),
  Output('hcc_gauge','figure'),
  Output('rvu_gauge','figure'),
  Output('ed_gauge','figure'),
  Output('qual_gauge','figure'),
  Output('survey_gauge','figure')],
  Input('provider_list', 'value'))
    
def update_graphs(prov):   # prov is chosen provider
#  cprov = str(prov)
 # return cprov
 # dplay = rvudata[prov]
 # fig = dplay.plot(kind="bar", title = cprov)
 # figrb = px.bar(dplay) # this is the bar chart showing the rvu for each week
  # call the function to get the gauge plots 

  #df = 
   # a dataframe with the rvu columns as index and values as only other column
  
  rvubr = px.bar(rvudata.loc[prov], title='Average Direct RVUs on Week Ending')
  qualbr = px.bar(qualdata.loc[prov], title='Quality Metrics')
  exclude = ['Experience of Care (Weighted Average)']
  survbr = px.bar(survdata.loc[prov,survdata.columns.difference(exclude)], title='Experience of Care')
  metricvalue = rvudata.loc[prov,'Average RVU/Day']
  plotcuts = gauge_tick_measures(rvucuts)
  title = "Direct RVUs per day"
  rvu_gauge_plot = gaugeplot(metricvalue,plotcuts,title)

  metricvalue = hccdata.loc[prov,'Recapture Rate']
  plotcuts = gauge_tick_measures(hcccuts)
  title = "Recapture Rate"
  hcc_gauge_plot = gaugeplot(metricvalue,plotcuts,title)

  metricvalue = eddata.loc[prov,'Percent Reduction in ED use']
  plotcuts = gauge_tick_measures(edcuts)
  title = "ED Use Reduction"
  ed_gauge_plot = gaugeplot(metricvalue,plotcuts,title)

  metricvalue = qualdata.loc[prov,'Overall']
  plotcuts = gauge_tick_measures(qualcuts)
  title = "Quality Measures"
  qual_gauge_plot = gaugeplot(metricvalue,plotcuts,title)

  metricvalue = survdata.loc[prov,'Experience of Care (Weighted Average)']
  plotcuts = gauge_tick_measures(survcuts)
  title = "Press Ganey"
  survey_gauge_plot = gaugeplot(metricvalue,plotcuts,title)

  return rvubr, qualbr, survbr, hcc_gauge_plot, rvu_gauge_plot, ed_gauge_plot, qual_gauge_plot, survey_gauge_plot




In [ ]:
from pyngrok import ngrok

In [ ]:

#! ngrok authtoken 2B3LmBt0LLziQsaWSJ3CBcuptL6_32EVfc6NMUwnp5ZMqeMvg
#public_url = ngrok.connect(port = '8050')
#public_url

In [ ]:
app.run_server()


Dash app running on:


<IPython.core.display.Javascript object>